In [1]:
import pandas as pd
import os
import numpy as np
import csv
from datetime import datetime, timedelta

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata

In [2]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/FINAL_PROJECT/DATASET/stock_tweets.csv'
df = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
print(df.shape)
df.head()

(80793, 4)


,Date,Tweet,Stock Name,Company Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."


In [4]:
sent_df = df.copy()
sent_df["score"] = ''
sent_df.head()

,Date,Tweet,Stock Name,Company Name,score
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",


In [5]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [6]:
%%time
sentiment_analyzer = SentimentIntensityAnalyzer()
for indx, row in sent_df.iterrows():
    try:
        sentence_i = unicodedata.normalize('NFKD', row['Tweet'])
        sentence_sentiment = sentiment_analyzer.polarity_scores(sentence_i)
        sent_df.at[indx, 'score'] = sentence_sentiment['compound']
    except TypeError:
        print(sent_df.loc[indx, 'Tweet'])
        print(indx)
        break

CPU times: user 44.9 s, sys: 146 ms, total: 45.1 s
Wall time: 52.1 s


In [7]:
sent_df.head()

,Date,Tweet,Stock Name,Company Name,score
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",0.0772
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",0.0
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",0.296
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",-0.7568
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",-0.875


In [8]:
# CONVERT DATETIME TO DATE

sent_df['Date'] = pd.to_datetime(sent_df['Date'])
sent_df['Date'] = sent_df['Date'].dt.date
sent_df.info()

# .dt.date extracts only the date values from the Date column
# to_datetime returns a datetime object to work on

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80793 entries, 0 to 80792
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          80793 non-null  object
 1   Tweet         80793 non-null  object
 2   Stock Name    80793 non-null  object
 3   Company Name  80793 non-null  object
 4   score         80793 non-null  object
dtypes: object(5)
memory usage: 3.1+ MB


In [9]:
# DROP COMPANY NAME

sent_df = sent_df.drop(columns=['Company Name'])
sent_df.head()

,Date,Tweet,Stock Name,score
0,2022-09-29,Mainstream media has done an amazing job at br...,TSLA,0.0772
1,2022-09-29,Tesla delivery estimates are at around 364k fr...,TSLA,0.0
2,2022-09-29,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,0.296
3,2022-09-29,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,-0.7568
4,2022-09-29,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,-0.875


In [10]:
# DROPPING TWWET COLUMN

sent_df = sent_df.drop(columns=['Tweet'])
sent_df.head()

,Date,Stock Name,score
0,2022-09-29,TSLA,0.0772
1,2022-09-29,TSLA,0.0
2,2022-09-29,TSLA,0.296
3,2022-09-29,TSLA,-0.7568
4,2022-09-29,TSLA,-0.875


In [11]:
#twitter_df = sent_df.groupby(sent_df['Date'], as_index=False).mean()
#print(twitter_df.shape)
#twitter_df.head()
twitter_df = sent_df.groupby([sent_df['Date'], sent_df['Stock Name']], as_index=False).mean()
print(twitter_df.shape)
twitter_df.head()

# as_index is False because otherwise date column becomes the index
# Stock Name is written in groupby so that we dont loose the column

(5910, 3)


,Date,Stock Name,score
0,2021-09-30,AAPL,0.09890
1,2021-09-30,AMD,0.41780
2,2021-09-30,AMZN,0.24648
3,2021-09-30,DIS,0.54610
4,2021-09-30,GOOG,0.44040


In [12]:
twitter_df.to_csv("/content/drive/MyDrive/FINAL_PROJECT/DATASET/TWEETS.csv", index=False)